In [1]:
import numpy as np

a, b = np.random.rand(2, 3)
a = a / np.linalg.norm(a) + np.random.rand()
b = b / np.linalg.norm(b) + np.random.rand()

A = np.asarray([a], dtype=float)
B = np.asarray([b], dtype=float)

In [2]:
from scipy.spatial.transform import Rotation

r = Rotation.align_vectors(A, B)[0]
print(r.as_matrix())

[[ 0.97834134 -0.11357885  0.17305509]
 [ 0.06520766  0.96256161  0.26310286]
 [-0.1964591  -0.24611989  0.94912003]]


In [3]:
print(r.apply(B), a)

[[0.72442534 0.55764021 0.91079515]] [1.37226962 1.05633067 1.72530755]


In [4]:
from ase.data import covalent_radii as COV_R
from ase.build import molecule
from ase.io import read
from ase import Atoms

core = [303, 334, 464]  # vertex fcc hollow
atoms = Atoms(read("../tests/OctCu10.xyz"))
adsorbate = molecule("CO")
adsorbate_index = 0


cov_radii_core = np.mean(COV_R[atoms.numbers[core]])

com_atoms = atoms.get_center_of_mass()
com_core = Atoms(atoms[core]).get_center_of_mass()
direction: np.ndarray = com_core - com_atoms
direction /= np.linalg.norm(direction)

com_ads = adsorbate.get_center_of_mass()
if len(adsorbate) == 0:
    raise ValueError("The adsorbate must have at least one atom.")
elif len(adsorbate) == 1:
    adsorbate_index = 0
else:
    if adsorbate_index is None:
        v2com_ads = adsorbate.positions - com_ads
        d2com_ads = np.linalg.norm(v2com_ads, axis=1)
        adsorbate_index = int(np.argmin(d2com_ads))
assert isinstance(adsorbate_index, int), (
    "The adsorbate_index must be None or integer."
)
ref_pos = adsorbate.positions[adsorbate_index]
B = np.asarray([ref_pos, com_ads])


_d2ref = COV_R[adsorbate.numbers[adsorbate_index]] + cov_radii_core
_d2com = float(np.linalg.norm(ref_pos - com_ads)) + _d2ref
target_ref_pos = com_core + _d2ref * direction
target_com_ads = com_core + _d2com * direction
A = np.asarray([target_ref_pos, target_com_ads])

In [5]:
from scipy.spatial.distance import pdist

print(pdist(A), pdist(B))
print(A, "\n", B)

[0.49327861] [0.49327861]
[[ 0.68074025  0.68074025 17.01850625]
 [ 0.7004399   0.7004399  17.5109975 ]] 
 [[ 0.00000000e+00  0.00000000e+00  4.93003000e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.75605498e-04]]


In [6]:
import sys

sys.path.append("..")

from adsorption.rotation import kabsch

r, t, rmsd = kabsch(A, B)

In [7]:
from adsorption.rotation import rotate

rotate(B, r) + t

array([[ 0.68074025,  0.68074025, 17.01850625],
       [ 0.7004399 ,  0.7004399 , 17.5109975 ]])